In [1]:
import numpy as np
import pandas as pd
import csv_figures
from csv_figures import add_time_col
from csv_figures import annual_Norm
import matplotlib.pyplot as plt
import requests
from matplotlib.widgets import CheckButtons

In [2]:
Naknek = pd.read_excel('../../Cleaned_data/Nakenk/Alaska_Naknek_cleaned.xlsx')

In [3]:
Naknek_new = add_time_col(Naknek)
Nak_annual_values,Nak_month = annual_Norm(Naknek_new)

/Users/guge/Desktop/acep-solar/new/ACEP_solar/GEGU/figures/csv_figures.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dataframe['Date'][i] = str(dataframe['Year'][i]) + '-' + str(dataframe['Month'][i])


In [4]:
# Get the data from the PV Watts --TMY2
list_parameters = {"formt": 'JSON', "api_key": "spJFj2l5ghY5jwk7dNfVYs3JHbpR6BOGHQNO8Y9Z", "system_capacity": 80, "module_type": 0, "losses": 14.08,
              "array_type": 0, "tilt": 50, "azimuth": 180, "lat": 61.20, "lon": -149.73, "dataset": 'tmy2'}
json_response = requests.get("https://developer.nrel.gov/api/pvwatts/v6", params = list_parameters).json()
TMY2 = pd.DataFrame(data = json_response['outputs'])

# Get the data from the PV Watts --TMY2
list_parameters = {"formt": 'JSON', "api_key": "spJFj2l5ghY5jwk7dNfVYs3JHbpR6BOGHQNO8Y9Z", "system_capacity": 80, "module_type": 0, "losses": 14.08,
              "array_type": 0, "tilt": 50, "azimuth": 180, "lat": 58.78, "lon": -157.01, "dataset": 'tmy3'}
json_response = requests.get("https://developer.nrel.gov/api/pvwatts/v6", params = list_parameters).json()
TMY3 = pd.DataFrame(data = json_response['outputs'])

In [5]:
new_Nak = pd.DataFrame({'Date':Nak_month,'Naknek':Nak_annual_values})


In [6]:
#new_Nak['TMY2'] = TMY2['ac_monthly']
#new_Nak['TMY3'] = TMY3['ac_monthly']
#Anchorage_16.drop(['TMY2_annual','TMY3_annual'],axis = 1,inplace = True)
TMY2_low = TMY2['ac_annual'][2]*0.95/80
TMY2_high = TMY2['ac_annual'][2]*1.05/80

TMY3_low = TMY3['ac_annual'][2]*0.95/80
TMY3_high = TMY3['ac_annual'][2]*1.05/80

new_Nak['TMY2_low'] = TMY2_low
new_Nak['TMY2_high'] = TMY2_high
new_Nak['TMY3_low'] = TMY3_low
new_Nak['TMY3_high'] = TMY3_high

new_Nak

,Date,Naknek,TMY2_low,TMY2_high,TMY3_low,TMY3_high
0,2018-02-01,680.245000,877.099932,969.42624,825.950781,912.892969
1,2018-03-01,698.947000,877.099932,969.42624,825.950781,912.892969
2,2018-04-01,649.477500,877.099932,969.42624,825.950781,912.892969
3,2018-05-01,658.896125,877.099932,969.42624,825.950781,912.892969
4,2018-06-01,669.220750,877.099932,969.42624,825.950781,912.892969
5,2018-07-01,670.092375,877.099932,969.42624,825.950781,912.892969
6,2018-08-01,648.458125,877.099932,969.42624,825.950781,912.892969
7,2018-09-01,630.214125,877.099932,969.42624,825.950781,912.892969
8,2018-10-01,647.980250,877.099932,969.42624,825.950781,912.892969
9,2018-11-01,650.384750,877.099932,969.42624,825.950781,912.892969


In [7]:
%matplotlib qt5
fig, ax = plt.subplots(figsize = (15, 8))
x = new_Nak['Date']


y_Nak = new_Nak['Naknek']
l0, = ax.plot(x ,y_Nak,label='Naknek')

y_TMY2_low = new_Nak['TMY2_low']
l1, = ax.plot(x,y_TMY2_low,'--',label = 'TMY2_low')

y_TMY2_high = new_Nak['TMY2_high']
l2, = ax.plot(x,y_TMY2_high,'--',label = 'TMY2_high')

y_TMY3_low = new_Nak['TMY3_low']
l3, = ax.plot(x,y_TMY3_low,'-.',label = 'TMY3_low')

y_TMY3_high = new_Nak['TMY3_high']
l4, = ax.plot(x,y_TMY3_high,'-.',label = 'TMY3_high')


ax.set_title("Rolling 12-Month Average of Produced Power, Over Time", size = 18)
ax.set_xlabel("Month", size = 14)
ax.set_ylabel("AC Power Produced (kWh) per DC Power Installed (kW)", size = 14)
ax.legend()


plt.subplots_adjust(left=0.2)
lines = [l0, l1, l2, l3, l4]

# Make checkbuttons with all plotted lines with correct visibility
rax = plt.axes([0.005, 0.48, 0.15, 0.4])
labels = [str(line.get_label()) for line in lines]
visibility = [line.get_visible() for line in lines]
check = CheckButtons(rax, labels, visibility)


def func(label):
    index = labels.index(label)
    lines[index].set_visible(not lines[index].get_visible())
    plt.draw()

check.on_clicked(func)


# Some 

plt.show()

#Ok,this seems to be fine,maybe we will talk about it after.

/miniconda3/lib/python3.7/site-packages/pandas/plotting/_converter.py:129: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)
